In [ ]:
import marimo as mo

In [ ]:
import os
from pathlib import Path
from pprint import pprint
from typing import Self
import datetime
import random
import dotenv
import pandas
import pydantic
import plotly.express
import altair
from ortools.sat.python import cp_model
import highspy
import amplify_sched
import didppy

# ジョブショップスケジューリング問題

$J || C_{\max}$ と書く.

- ジョブ $J_1, \dots, J_n$
- ジョブ $J_j$ に属するオペレーション $O_{1j}, \dots, O_{m_jj}$. この順で処理される.
- 機械 $M_1, \dots, M_m$
- オペレーション $O_{ij}$ は機械 $\mu_{ij}$ で作業時間 $p_{ij}$ かけて処理する.
- オペレーションは中断できない
- 最後のオペレーションの終了時刻を最小化

In [ ]:
parent = str(Path(os.path.abspath(__file__)).parent)
data_dir = os.path.join(parent, "data")

In [ ]:
class Task(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)

    machine: int = pydantic.Field(..., ge=0, frozen=True)
    time: int = pydantic.Field(..., ge=0, frozen=True)

In [ ]:
class Job(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)

    tasks: list[Task] = pydantic.Field(frozen=True)

    def from_file(fname: str) -> list[Self]:
        with open(fname) as f:
            n, m = None, None
            machine, proc_time = {}, {}

            i = 0
            for line in f:
                if line[0] == "#":
                    continue

                if n is None or m is None:
                    n, m = map(int, line.split())
                    print(f"{n=}, {m=}")
                    continue

                L = list(map(int, line.split()))
                for j in range(m):
                    machine[i, j] = L[2 * j]
                    proc_time[i, j] = L[2 * j + 1]
                i += 1

        jobs = []
        for i in range(n):
            tasks = []
            for j in range(m):
                tasks.append(Task(machine=machine[i, j], time=proc_time[i, j]))
            jobs.append(Job(tasks=tasks))

        return jobs

In [ ]:
def plot_plotly(df: pandas.DataFrame):
    return plotly.express.timeline(
        df,
        x_start="start",
        x_end="end",
        y="resource",
        color="job",
        opacity=0.5,
    ).update_yaxes(categoryorder="category descending")

In [ ]:
def plot_altair(df: pandas.DataFrame):
    return (
        altair.Chart(df)
        .mark_bar()
        .encode(
            x="start",
            x2="end",
            y="resource",
            color="job",
        )
        .properties(width="container", height=400)
    )

In [ ]:
fname1 = os.path.join(data_dir, "ft06.txt")
jobs1 = Job.from_file(fname1)
pprint(jobs1)

n=6, m=6
[Job(tasks=[Task(machine=2, time=1), Task(machine=0, time=3), Task(machine=1, time=6), Task(machine=3, time=7), Task(machine=5, time=3), Task(machine=4, time=6)]),
 Job(tasks=[Task(machine=1, time=8), Task(machine=2, time=5), Task(machine=4, time=10), Task(machine=5, time=10), Task(machine=0, time=10), Task(machine=3, time=4)]),
 Job(tasks=[Task(machine=2, time=5), Task(machine=3, time=4), Task(machine=5, time=8), Task(machine=0, time=9), Task(machine=1, time=1), Task(machine=4, time=7)]),
 Job(tasks=[Task(machine=1, time=5), Task(machine=0, time=5), Task(machine=2, time=5), Task(machine=3, time=3), Task(machine=4, time=8), Task(machine=5, time=9)]),
 Job(tasks=[Task(machine=2, time=9), Task(machine=1, time=3), Task(machine=4, time=5), Task(machine=5, time=4), Task(machine=0, time=3), Task(machine=3, time=1)]),
 Job(tasks=[Task(machine=1, time=3), Task(machine=3, time=3), Task(machine=5, time=9), Task(machine=0, time=10), Task(machine=4, time=4), Task(machine=2, time=1)])]


## OR-Tools による求解

In [ ]:
class ModelCpSat:
    def __init__(self, jobs: list[Job]):
        self.jobs = jobs
        self.model = cp_model.CpModel()
        num_machines = len(
            set(task.machine for job in self.jobs for task in job.tasks)
        )
        self.machines = list(range(num_machines))
        horizon = sum(task.time for job in self.jobs for task in job.tasks)

        self.starts = [[None for task in job.tasks] for job in jobs]
        self.intervals = [[None for task in job.tasks] for job in jobs]
        machine_to_interval = {m: [] for m in self.machines}

        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                suffix = f"_{id_job}_{id_task}"
                start = self.model.new_int_var(0, horizon, "start" + suffix)
                interval = self.model.new_fixed_size_interval_var(
                    start, task.time, "interval" + suffix
                )
                self.starts[id_job][id_task] = start
                self.intervals[id_job][id_task] = interval
                machine_to_interval[task.machine].append(interval)

        for machine in machine_to_interval:
            if len(machine_to_interval[machine]) > 0:
                self.model.add_no_overlap(machine_to_interval[machine])

        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                if id_task > 0:
                    curr = self.intervals[id_job][id_task]
                    prev = self.intervals[id_job][id_task - 1]
                    self.model.add(curr.start_expr() >= prev.end_expr())

        makespan = self.model.new_int_var(0, horizon, "makespan")
        self.model.add_max_equality(
            makespan,
            [
                self.intervals[id_job][-1].end_expr()
                for id_job, job in enumerate(self.jobs)
            ],
        )
        self.model.minimize(makespan)

    def solve(self, timeout: int = 10):
        self.solver = cp_model.CpSolver()
        self.solver.parameters.log_search_progress = True
        self.solver.parameters.max_time_in_seconds = timeout
        self.status = self.solver.solve(self.model)

    def to_df(self) -> pandas.DataFrame:
        today = datetime.date.today()
        l = []
        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                start = self.solver.value(
                    self.intervals[id_job][id_task].start_expr()
                )
                end = start + self.jobs[id_job].tasks[id_task].time
                l.append(
                    dict(
                        job=f"job{id_job}",
                        task=f"task{id_task}",
                        resource=f"machine{self.jobs[id_job].tasks[id_task].machine}",
                        start=today + datetime.timedelta(start),
                        end=today + datetime.timedelta(end),
                    )
                )
        df = pandas.DataFrame(l)
        df["start"] = pandas.to_datetime(df["start"])
        df["end"] = pandas.to_datetime(df["end"])
        return df

In [ ]:
model1_cpsat = ModelCpSat(jobs1)
model1_cpsat.solve()


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 10 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x43f0af3eddfba1f7)
#Variables: 37 (#ints: 1 in objective) (36 primary variables)
  - 37 in [0,197]
#kInterval: 36
#kLinMax: 1 (#expressions: 6)
#kLinear2: 30
#kNoOverlap: 6 (#intervals: 36)

Starting presolve at 0.00s
  1.57e-05s  0.00e+00d  [DetectDominanceRelations] 
  2.80e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=7 #num_dual_strengthening=1 
  5.12e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  3.80e-06s  0.00e+00d  [DetectDuplicateColumns] 
  1.54e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 146 nodes and 175 arcs.
[Symmetry] Symmetry computation done. time: 1.8075e-05 dtime: 1.547e-05
  1.42e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.77e-04s  8.14e-07d  [Probe] 
  1.68e-06s  0.00e+00d  [MaxClique] 
  1.91e-05s  0.00e+00d  [De

In [ ]:
plot_plotly(model1_cpsat.to_df())

<marimo-plotly data-figure='{"data": [{"base": ["2025-06-24T00:00:00.000000000", "2025-06-25T00:00:00.000000000", "2025-07-05T00:00:00.000000000", "2025-07-19T00:00:00.000000000", "2025-07-27T00:00:00.000000000", "2025-07-31T00:00:00.000000000"], "hovertemplate": "job=job0<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job0", "marker": {"color": "#636efa", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job0", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AFwmBQAUcw8AKOYeAIQMJAAUcw8AKOYe"}, "xaxis": "x", "y": ["machine2", "machine0", "machine1", "machine3", "machine5", "machine4"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-19T00:00:00.000000000", "2025-06-27T00:00:00.000000000", "2025-07-02T00:00:00.000000000", "2025-07-17T00:00:00.000000000", "2025-07-27T00:00:00.000000000", "2025-08-06T00:00:00.000000000"], "hovertemplate": "job=job1<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job1", "marker": {"color": "#EF553B", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job1", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AOAyKQDMvxkAmH8zAJh/MwCYfzMAcJkU"}, "xaxis": "x", "y": ["machine1", "machine2", "machine4", "machine5", "machine0", "machine3"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-19T00:00:00.000000000", "2025-06-24T00:00:00.000000000", "2025-06-28T00:00:00.000000000", "2025-07-07T00:00:00.000000000", "2025-07-16T00:00:00.000000000", "2025-08-06T00:00:00.000000000"], "hovertemplate": "job=job2<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job2", "marker": {"color": "#00cc96", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job2", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AMy/GQBwmRQA4DIpADxZLgBcJgUAhAwk"}, "xaxis": "x", "y": ["machine2", "machine3", "machine5", "machine0", "machine1", "machine4"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-27T00:00:00.000000000", "2025-07-02T00:00:00.000000000", "2025-07-11T00:00:00.000000000", "2025-07-16T00:00:00.000000000", "2025-07-19T00:00:00.000000000", "2025-08-03T00:00:00.000000000"], "hovertemplate": "job=job3<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job3", "marker": {"color": "#ab63fa", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job3", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AMy/GQDMvxkAzL8ZABRzDwDgMikAPFku"}, "xaxis": "x", "y": ["machine1", "machine0", "machine2", "machine3", "machine4", "machine5"], "yaxis": "y", "type": "bar"}, {"base": ["2025-07-02T00:00:00.000000000", "2025-07-11T00:00:00.000000000", "2025-07-14T00:00:00.000000000", "2025-07-30T00:00:00.000000000", "2025-08-06T00:00:00.000000000", "2025-08-10T00:00:00.000000000"], "hovertemplate": "job=job4<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job4", "marker": {"color": "#FFA15A", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job4", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "ADxZLgAUcw8AzL8ZAHCZFAAUcw8AXCYF"}, "xaxis": "x", "y": ["machine2", "machine1", "machine4", "machine5", "machine0", "machine3"], "yaxis": "y", "type": "bar"}, {"base": ["2025-07-02T00:00:00.000000000", "2025-07-05T00:00:00.000000000", "2025-07-08T00:00:00.000000000", "2025-07-17T00:00:00.000000000", "2025-07-27T00:00:00.000000000", "2025-07-31T00:00:00.000000000"], "hovertemplate": "job=job5<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job5", "marker": {"color": "#19d3f3", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job5", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "ABRzDwAUcw8APFkuAJh/MwBwmRQAXCYF"}, "xaxis": "x", "y": ["machine1", "machine3", "machine5", "machine0", "machine4", "machine2

In [ ]:
mo.ui.altair_chart(plot_altair(model1_cpsat.to_df()))

<marimo-vega data-initial-value='{}' data-label='null' data-spec='{"config": {"view": {"continuousWidth": 300, "continuousHeight": 300}}, "data": {"url": "./@file/5658-1220480-zwMztQiF.arrow", "format": {"type": "arrow"}}, "mark": {"type": "bar"}, "encoding": {"color": {"field": "job", "type": "nominal"}, "x": {"field": "start", "type": "temporal"}, "x2": {"field": "end"}, "y": {"field": "resource", "type": "nominal"}}, "height": 400, "width": "container", "$schema": "https://vega.github.io/schema/vega-lite/v5.20.1.json"}' data-chart-selection='true' data-field-selection='true'>

In [ ]:
plot_altair(model1_cpsat.to_df())

## 数理最適化ソルバーによる求解

各ジョブに含まれるオペレーション数は機械の数 $m$ に一致すると仮定する.

\begin{align*}
&\min &z \\
&\text{s.t. } & s_{ij} + p_{ij} - M (1 - x_{ijkl}) &\leq s_{kl} \quad &(\forall j \ne k) \\
& & x_{ijkl} + x_{klij} &= 1 \quad &((i,j) \ne (k,l) \land \text{machine} (i,j) = \text{machine} (k,l)) \\
& & s_{ij} + p_{ij} &\le s_{i,j+1} \quad &(\forall i, j = 1, \dots, m-1) \\
& & s_{im} &\le z \quad &(\forall i) \\
& & s_{i1} &\ge 0 \quad &(\forall i) \\
& & x_{ijkl} &\in \{ 0, 1 \} \quad &(\forall (i,j) \ne (k,l))
\end{align*}

In [ ]:
class _MyInterval:
    def __init__(self, model: highspy.Highs, lb: int, ub: int, proctime: int):
        self.lb = lb
        self.ub = ub
        self.start = model.addVariable(lb=lb, ub=ub - proctime)
        self.time = proctime
        self.end = self.start + self.time


def _my_add_no_overlap(model: highspy.Highs, tasks: list[_MyInterval]) -> None:
    for idx1, task1 in enumerate(tasks):
        for idx2, task2 in enumerate(tasks):
            if idx1 >= idx2:
                continue

            big_m = max(task1.ub - task2.lb, task2.ub - task1.lb)
            tmp1 = model.addBinary()  # [ task1 ] [ task2 ] の順
            tmp2 = model.addBinary()  # [ task2 ] [ task1 ] の順
            model.addConstrs(
                [
                    task1.end - big_m * (1 - tmp1) <= task2.start,
                    task2.end - big_m * (1 - tmp2) <= task1.start,
                    tmp1 + tmp2 == 1,
                ]
            )


class ModelHighs:
    def __init__(self, jobs: list[Job]):
        self.jobs = jobs
        num_machines = len(
            set(task.machine for job in self.jobs for task in job.tasks)
        )
        self.machines = list(range(num_machines))

        self.model = highspy.Highs()

        self.intervals = [[None for task in job.tasks] for job in jobs]
        machine_to_interval = {m: [] for m in self.machines}

        horizon = sum(task.time for job in self.jobs for task in job.tasks)
        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                interval = _MyInterval(self.model, 0, horizon, task.time)
                self.intervals[id_job][id_task] = interval
                machine_to_interval[task.machine].append(interval)

        for machine in machine_to_interval:
            if len(machine_to_interval[machine]) > 0:
                _my_add_no_overlap(self.model, machine_to_interval[machine])

        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                if id_task > 0:
                    curr = self.intervals[id_job][id_task]
                    prev = self.intervals[id_job][id_task - 1]
                    self.model.addConstr(curr.start >= prev.end)

        makespan = self.model.addVariable(lb=0, ub=horizon)
        self.model.addConstrs(
            [
                self.intervals[id_job][-1].end <= makespan
                for id_job, job in enumerate(self.jobs)
            ],
        )
        self.model.minimize(makespan)

    def solve(self) -> None:
        self.model.run()
        self.solution = self.model.getSolution()

    def to_df(self) -> pandas.DataFrame:
        today = datetime.date.today()
        l = []
        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                start = self.solution.col_value[
                    self.intervals[id_job][id_task].start.index
                ]
                start = round(start)
                end = start + self.jobs[id_job].tasks[id_task].time
                l.append(
                    dict(
                        job=f"job{id_job}",
                        task=f"task{id_task}",
                        resource=f"machine{self.jobs[id_job].tasks[id_task].machine}",
                        start=today + datetime.timedelta(start),
                        end=today + datetime.timedelta(end),
                    )
                )
        df = pandas.DataFrame(l)
        df["start"] = pandas.to_datetime(df["start"])
        df["end"] = pandas.to_datetime(df["end"])
        return df

In [ ]:
model1_highs = ModelHighs(jobs1)
model1_highs.solve()

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 306 rows; 217 cols; 792 nonzeros; 180 integer variables (180 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+02]
  RHS    [1e+00, 2e+02]
Presolving model
216 rows, 127 cols, 612 nonzeros  0s
212 rows, 127 cols, 600 nonzeros  0s

Solving MIP model with:
   212 rows
   127 cols (90 binary, 0 integer, 0 implied int., 37 continuous, 0 domain fixed)
   600 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   9               inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   47              152               69.08%        0      0      0        92     0.0s


 L       0       0         0   0.00%   49.48911212     60                17.52%     2421    188      0      1007     0.2s

6.7% inactive integer columns, restarting
Model after restart has 186 rows, 109 cols (72 bin., 0 int., 0 impl., 37 cont., 0 dom.fix.), and 510 nonzeros

         0       0         0   0.00%   49.78946174     60                17.02%       45      0      0      2085     0.2s
         0       0         0   0.00%   49.7947358      60                17.01%       45     26      0      2190     0.2s


 L       0       0         0   0.00%   49.98814373     57                12.30%     1682    133      0      2786     0.2s


 B     254       8        99  93.21%   51.60271405     56                 7.85%     3930    137    520      8738     0.5s


 B     272       6       105  94.92%   51.73613877     55                 5.93%     3726     57    560      9288     0.5s


       286       0       113 100.00%   55              55                 0.00%     4191     37    597      9803     0.5s

Solving report
  Status            Optimal
  Primal bound      55
  Dual bound        55
  Gap               0% (tolerance: 0.01%)
  P-D integral      0.143436718592
  Solution status   feasible
                    55 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.54 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 2
  Nodes             286
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     9803 (total)
                    1516 (strong br.)
                    2336 (separation)
                    1486 (heuristics)
MIP  has 306 rows; 217 cols; 792 nonzeros; 180 integer variables (180 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00


Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   47              55                14.55%        0      0      0         0     0.0s
         0       0         0   0.00%   47              55                14.55%        0      0      0        84     0.0s



26.0% inactive integer columns, restarting
         0       0         0   0.00%   55              55                 0.00%        0    134      0       227     0.0s

Solving report
  Status            Optimal
  Primal bound      55
  Dual bound        55
  Gap               0% (tolerance: 0.01%)
  P-D integral      0.00501451666647
  Solution status   feasible
                    55 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.05 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 1
  Nodes             0
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     227 (total)
                    0 (strong br.)
                    52 (separation)
                    91 (heuristics)


In [ ]:
plot_altair(model1_highs.to_df())

## FIXSTARS Amplify Scheduling Engine による求解

In [ ]:
dotenv.load_dotenv(dotenv.find_dotenv(usecwd=True))
token = os.environ["FIXSTARS_SE"]


class ModelAmplifySe:
    def __init__(self, jobs: list[Job]):
        self.jobs = jobs
        num_machines = len(
            set(task.machine for job in self.jobs for task in job.tasks)
        )
        self.machines = list(range(num_machines))
        self.se_machines = [
            amplify_sched.Machine(name=f"machine{midx}")
            for midx in self.machines
        ]

        self.model = amplify_sched.Model()

        for semachine in self.se_machines:
            self.model.machines.add(machine=semachine)

        self.se_jobs = [
            amplify_sched.Job(name=f"job{jidx}")
            for jidx, _ in enumerate(self.jobs)
        ]
        for idx, job in enumerate(self.jobs):
            sejob = self.se_jobs[idx]
            self.model.jobs.add(sejob)
            for jdx, task in enumerate(job.tasks):
                semachine = self.se_machines[task.machine]
                setask = amplify_sched.Task()
                setask.processing_times[semachine] = task.time
                self.model.jobs[sejob.name].append(setask)

    def solve(self, timeout: int = 5) -> None:
        self.solution = self.model.solve(token=token, timeout=timeout)

    def get_makespan(self) -> int:
        return int(self.solution.table["Finish"].max())

    def to_df(self) -> pandas.DataFrame:
        sol_df = self.solution.table
        today = datetime.date.today()
        l = []
        for id_job, job in enumerate(self.jobs):
            sejob = self.se_jobs[id_job]
            for id_task, task in enumerate(job.tasks):
                start = int(
                    sol_df[sol_df["Job"] == sejob.name]["Start"].reset_index(
                        drop=True
                    )[id_task]
                )
                end = start + self.jobs[id_job].tasks[id_task].time
                l.append(
                    dict(
                        job=f"job{id_job}",
                        task=f"task{id_task}",
                        resource=f"machine{self.jobs[id_job].tasks[id_task].machine}",
                        start=today + datetime.timedelta(start),
                        end=today + datetime.timedelta(end),
                    )
                )
        df = pandas.DataFrame(l)
        df["start"] = pandas.to_datetime(df["start"])
        df["end"] = pandas.to_datetime(df["end"])
        return df

In [ ]:
model1_amplify = ModelAmplifySe(jobs1)
model1_amplify.solve()

print(f"makespan = {model1_amplify.get_makespan()}")

makespan = 55


In [ ]:
model1_amplify.solution.timeline(machine_view=True)

<marimo-plotly data-figure='{"data": [{"base": [5, 6, 16, 30, 38, 42], "customdata": [["job0", 0], ["job0", 1], ["job0", 2], ["job0", 3], ["job0", 4], ["job0", 5]], "hovertemplate": "Job=%{customdata[0]}<br>Start=%{base}<br>Finish=%{x}<br>Machine=%{y}<br>Process=%{customdata[1]}<extra></extra>", "legendgroup": "job0", "marker": {"color": "#636efa", "pattern": {"shape": ""}}, "name": "job0", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i1", "bdata": "AQMGBwMG"}, "xaxis": "x", "y": ["machine2", "machine0", "machine1", "machine3", "machine5", "machine4"], "yaxis": "y", "type": "bar"}, {"base": [0, 8, 13, 28, 38, 48], "customdata": [["job1", 0], ["job1", 1], ["job1", 2], ["job1", 3], ["job1", 4], ["job1", 5]], "hovertemplate": "Job=%{customdata[0]}<br>Start=%{base}<br>Finish=%{x}<br>Machine=%{y}<br>Process=%{customdata[1]}<extra></extra>", "legendgroup": "job1", "marker": {"color": "#EF553B", "pattern": {"shape": ""}}, "name": "job1", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i1", "bdata": "CAUKCgoE"}, "xaxis": "x", "y": ["machine1", "machine2", "machine4", "machine5", "machine0", "machine3"], "yaxis": "y", "type": "bar"}, {"base": [0, 5, 9, 18, 27, 48], "customdata": [["job2", 0], ["job2", 1], ["job2", 2], ["job2", 3], ["job2", 4], ["job2", 5]], "hovertemplate": "Job=%{customdata[0]}<br>Start=%{base}<br>Finish=%{x}<br>Machine=%{y}<br>Process=%{customdata[1]}<extra></extra>", "legendgroup": "job2", "marker": {"color": "#00cc96", "pattern": {"shape": ""}}, "name": "job2", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i1", "bdata": "BQQICQEH"}, "xaxis": "x", "y": ["machine2", "machine3", "machine5", "machine0", "machine1", "machine4"], "yaxis": "y", "type": "bar"}, {"base": [8, 13, 22, 27, 30, 45], "customdata": [["job3", 0], ["job3", 1], ["job3", 2], ["job3", 3], ["job3", 4], ["job3", 5]], "hovertemplate": "Job=%{customdata[0]}<br>Start=%{base}<br>Finish=%{x}<br>Machine=%{y}<br>Process=%{customdata[1]}<extra></extra>", "legendgroup": "job3", "marker": {"color": "#ab63fa", "pattern": {"shape": ""}}, "name": "job3", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i1", "bdata": "BQUFAwgJ"}, "xaxis": "x", "y": ["machine1", "machine0", "machine2", "machine3", "machine4", "machine5"], "yaxis": "y", "type": "bar"}, {"base": [13, 22, 25, 41, 48, 52], "customdata": [["job4", 0], ["job4", 1], ["job4", 2], ["job4", 3], ["job4", 4], ["job4", 5]], "hovertemplate": "Job=%{customdata[0]}<br>Start=%{base}<br>Finish=%{x}<br>Machine=%{y}<br>Process=%{customdata[1]}<extra></extra>", "legendgroup": "job4", "marker": {"color": "#FFA15A", "pattern": {"shape": ""}}, "name": "job4", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i1", "bdata": "CQMFBAMB"}, "xaxis": "x", "y": ["machine2", "machine1", "machine4", "machine5", "machine0", "machine3"], "yaxis": "y", "type": "bar"}, {"base": [13, 16, 19, 28, 38, 42], "customdata": [["job5", 0], ["job5", 1], ["job5", 2], ["job5", 3], ["job5", 4], ["job5", 5]], "hovertemplate": "Job=%{customdata[0]}<br>Start=%{base}<br>Finish=%{x}<br>Machine=%{y}<br>Process=%{customdata[1]}<extra></extra>", "legendgroup": "job5", "marker": {"color": "#19d3f3", "pattern": {"shape": ""}}, "name": "job5", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i1", "bdata": "AwMJCgQB"}, "xaxis": "x", "y": ["machine1", "machine3", "machine5", "machine0", "machine4", "machine2"], "yaxis": "y", "type": "bar"}], "layout": {"template": {"data": {"histogram2dcontour": [{"type": "histogram2dcontour", "colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f

In [ ]:
plot_altair(model1_amplify.to_df())

## より大きい問題

In [ ]:
def gen_data(n_jobs: int, n_machines: int) -> list[Job]:
    random.seed(0)
    jobs = []
    for id_job in range(n_jobs):
        machines = list(range(n_machines))
        random.shuffle(machines)
        tasks = []
        for id_task in range(n_machines):
            machine = machines[id_task]
            time = random.randint(1, 10)
            tasks.append(Task(machine=machine, time=time))

        jobs.append(Job(tasks=tasks))

    return jobs

In [ ]:
jobs2 = gen_data(45, 15)
pprint(jobs2)

[Job(tasks=[Task(machine=1, time=9), Task(machine=10, time=3), Task(machine=9, time=5), Task(machine=5, time=3), Task(machine=11, time=2), Task(machine=2, time=10), Task(machine=3, time=5), Task(machine=7, time=9), Task(machine=8, time=10), Task(machine=4, time=3), Task(machine=0, time=5), Task(machine=14, time=2), Task(machine=12, time=2), Task(machine=6, time=6), Task(machine=13, time=8)]),
 Job(tasks=[Task(machine=13, time=2), Task(machine=11, time=7), Task(machine=10, time=1), Task(machine=0, time=10), Task(machine=2, time=8), Task(machine=4, time=6), Task(machine=14, time=4), Task(machine=7, time=6), Task(machine=3, time=2), Task(machine=9, time=4), Task(machine=12, time=10), Task(machine=6, time=4), Task(machine=5, time=4), Task(machine=1, time=3), Task(machine=8, time=9)]),
 Job(tasks=[Task(machine=10, time=10), Task(machine=6, time=9), Task(machine=3, time=10), Task(machine=11, time=5), Task(machine=0, time=8), Task(machine=2, time=2), Task(machine=9, time=10), Task(machine=4, 

In [ ]:
model2_cpsat = ModelCpSat(jobs2)
model2_cpsat.solve()


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 10 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x62b4a1d48ce14446)
#Variables: 676 (#ints: 1 in objective) (675 primary variables)
  - 676 in [0,3683]
#kInterval: 675
#kLinMax: 1 (#expressions: 45)
#kLinear2: 630
#kNoOverlap: 15 (#intervals: 675)

Starting presolve at 0.00s
  1.17e-04s  0.00e+00d  [DetectDominanceRelations] 
  6.51e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=16 #num_dual_strengthening=1 
  4.43e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  2.08e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.59e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'672 nodes and 3'331 arcs.
[Symmetry] Symmetry computation done. time: 0.000176384 dtime: 0.00031934
  1.63e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  9.79e-04s  4.47e-06d  [Probe] #new_bounds=1 
  3.25e-06s  0.00e+00d  [MaxClique

  1.53e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'672 nodes and 3'331 arcs.
[Symmetry] Symmetry computation done. time: 0.000164923 dtime: 0.00031934
  1.55e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  9.15e-04s  4.36e-06d  [Probe] 
  2.38e-06s  0.00e+00d  [MaxClique] 
  1.07e-04s  0.00e+00d  [DetectDominanceRelations] 
  9.23e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  7.91e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.53e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.51e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  6.11e-05s  4.05e-06d  [DetectDominatedLinearConstraints] #relevant_constraints=675 
  8.18e-05s  0.00e+00d  [DetectDifferentVariables] 
  3.55e-06s  0.00e+00d  [ProcessSetPPC] 
  3.84e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  1.73e-05s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  2.82e-05s  2.41e-05d  [FindBigVer

#Bound   0.07s best:inf   next:[184,3683] max_lp


#1       0.10s best:324   next:[184,323]  no_lp
#Bound   0.10s best:324   next:[297,323]  no_lp


#2       0.15s best:323   next:[297,322]  quick_restart_no_lp


...

#14      1.69s best:300   next:[297,299]  graph_var_lns (d=7.07e-01 s=37 t=0.10 p=1.00 stall=0 h=base)


#15      2.45s best:299   next:[297,298]  no_lp


#16      2.53s best:298   next:[297,297]  no_lp


#Model   2.55s var:675/676 constraints:1320/1365


#17      2.61s best:297   next:[]         no_lp
#Done    2.61s no_lp

Task timing                                  n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                       'default_lp':         1 [   2.59s,    2.59s]    2.59s   0.00ns    2.59s         1 [ 83.16ms,  83.16ms]  83.16ms   0.00ns  83.16ms
                 'feasibility_pump':         2 [  4.84ms,   5.38ms]   5.11ms 270.87us  10.21ms         1 [  1.07ms,   1.07ms]   1.07ms   0.00ns   1.07ms
                            'fixed':         1 [   2.59s,    2.59s]    2.59s   0.00ns    2.59s         1 [ 60.31ms,  60.31ms]  60.31ms   0.00ns  60.31ms
                               'fj':         1 [ 87.12ms,  87.12ms]  87.12ms   0.00ns  87.12ms         1 [100.26ms, 100.26ms] 100.26ms   0.00ns 100.26ms
                               'fj':         2 [ 56.78ms,  56.91ms]  56.85ms  62.96us 113.69ms         2 [100.48ms, 100.69ms] 100.58ms 104.73us 201.17ms
            

In [ ]:
plot_plotly(model2_cpsat.to_df())

<marimo-plotly data-figure='{"data": [{"base": ["2025-07-17T00:00:00.000000000", "2025-07-26T00:00:00.000000000", "2025-07-29T00:00:00.000000000", "2025-08-10T00:00:00.000000000", "2025-08-31T00:00:00.000000000", "2025-09-15T00:00:00.000000000", "2025-10-26T00:00:00.000000000", "2025-11-02T00:00:00.000000000", "2025-12-02T00:00:00.000000000", "2026-01-22T00:00:00.000000000", "2026-02-01T00:00:00.000000000", "2026-02-09T00:00:00.000000000", "2026-02-11T00:00:00.000000000", "2026-02-13T00:00:00.000000000", "2026-03-09T00:00:00.000000000"], "hovertemplate": "job=job0<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job0", "marker": {"color": "#636efa", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job0", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "ADxZLgAUcw8AzL8ZABRzDwC4TAoAmH8zAMy/GQA8WS4AmH8zABRzDwDMvxkAuEwKALhMCgAo5h4A4DIp"}, "xaxis": "x", "y": ["machine1", "machine10", "machine9", "machine5", "machine11", "machine2", "machine3", "machine7", "machine8", "machine4", "machine0", "machine14", "machine12", "machine6", "machine13"], "yaxis": "y", "type": "bar"}, {"base": ["2025-07-10T00:00:00.000000000", "2025-07-14T00:00:00.000000000", "2025-08-11T00:00:00.000000000", "2025-08-12T00:00:00.000000000", "2025-08-22T00:00:00.000000000", "2025-09-22T00:00:00.000000000", "2025-09-30T00:00:00.000000000", "2025-10-27T00:00:00.000000000", "2025-12-23T00:00:00.000000000", "2025-12-25T00:00:00.000000000", "2025-12-30T00:00:00.000000000", "2026-01-25T00:00:00.000000000", "2026-03-19T00:00:00.000000000", "2026-03-26T00:00:00.000000000", "2026-03-29T00:00:00.000000000"], "hovertemplate": "job=job1<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job1", "marker": {"color": "#EF553B", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job1", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "ALhMCgCEDCQAXCYFAJh/MwDgMikAKOYeAHCZFAAo5h4AuEwKAHCZFACYfzMAcJkUAHCZFAAUcw8APFku"}, "xaxis": "x", "y": ["machine13", "machine11", "machine10", "machine0", "machine2", "machine4", "machine14", "machine7", "machine3", "machine9", "machine12", "machine6", "machine5", "machine1", "machine8"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-19T00:00:00.000000000", "2025-07-02T00:00:00.000000000", "2025-07-22T00:00:00.000000000", "2025-08-16T00:00:00.000000000", "2025-08-29T00:00:00.000000000", "2025-10-02T00:00:00.000000000", "2025-10-15T00:00:00.000000000", "2025-11-12T00:00:00.000000000", "2025-12-01T00:00:00.000000000", "2025-12-19T00:00:00.000000000", "2026-01-19T00:00:00.000000000", "2026-03-13T00:00:00.000000000", "2026-03-29T00:00:00.000000000", "2026-04-04T00:00:00.000000000", "2026-04-08T00:00:00.000000000"], "hovertemplate": "job=job2<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job2", "marker": {"color": "#00cc96", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job2", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AJh/MwA8WS4AmH8zAMy/GQDgMikAuEwKAJh/MwCEDCQAKOYeAJh/MwBwmRQAzL8ZABRzDwBwmRQAFHMP"}, "xaxis": "x", "y": ["machine10", "machine6", "machine3", "machine11", "machine0", "machine2", "machine9", "machine4", "machine12", "machine14", "machine8", "machine5", "machine13", "machine1", "machine7"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-19T00:00:00.000000000", "2025-06-28T00:00:00.000000000", "2025-07-08T00:00:00.000000000", "2025-07-12T00:00:00.000000000", "2025-07-22T00:00:00.000000000", "2025-08-01T00:00:00.000000000", "2025-09-21T00:00:00.000000000", "2025-10-24T00:00:00.000000000", "2025-11-01T00:00:00.000000000", "2025-11-17T00:00:00.000000000", "2025-12-04T00:00:00.000000000", "2026-01-16T00:00:00.000000000", "2026-01-29T00:00:00.000000000", "2026-02-11T00:00:00.000000000", "2026-04-02T00:00:00.000000000"], "hovertemplate": "job=job3<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></ext

In [ ]:
# model2_highs = ModelHighs(jobs2)
# model2_highs.solve()

In [ ]:
# plot_altair(model2_highs.to_df())

In [ ]:
model2_amplify = ModelAmplifySe(jobs2)
model2_amplify.solve(timeout=5)

print(f"makespan = {model2_amplify.get_makespan()}")

makespan = 297


In [ ]:
plot_plotly(model2_amplify.to_df())

<marimo-plotly data-figure='{"data": [{"base": ["2025-07-05T00:00:00.000000000", "2025-07-31T00:00:00.000000000", "2025-08-05T00:00:00.000000000", "2025-08-12T00:00:00.000000000", "2025-09-12T00:00:00.000000000", "2025-09-18T00:00:00.000000000", "2025-10-18T00:00:00.000000000", "2025-10-24T00:00:00.000000000", "2025-12-14T00:00:00.000000000", "2025-12-29T00:00:00.000000000", "2026-01-01T00:00:00.000000000", "2026-02-09T00:00:00.000000000", "2026-02-11T00:00:00.000000000", "2026-02-13T00:00:00.000000000", "2026-03-16T00:00:00.000000000"], "hovertemplate": "job=job0<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job0", "marker": {"color": "#636efa", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job0", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "ADxZLgAUcw8AzL8ZABRzDwC4TAoAmH8zAMy/GQA8WS4AmH8zABRzDwDMvxkAuEwKALhMCgAo5h4A4DIp"}, "xaxis": "x", "y": ["machine1", "machine10", "machine9", "machine5", "machine11", "machine2", "machine3", "machine7", "machine8", "machine4", "machine0", "machine14", "machine12", "machine6", "machine13"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-29T00:00:00.000000000", "2025-07-14T00:00:00.000000000", "2025-08-15T00:00:00.000000000", "2025-08-16T00:00:00.000000000", "2025-08-26T00:00:00.000000000", "2025-09-22T00:00:00.000000000", "2025-09-30T00:00:00.000000000", "2025-10-04T00:00:00.000000000", "2025-12-23T00:00:00.000000000", "2025-12-26T00:00:00.000000000", "2025-12-31T00:00:00.000000000", "2026-01-15T00:00:00.000000000", "2026-02-23T00:00:00.000000000", "2026-02-27T00:00:00.000000000", "2026-03-02T00:00:00.000000000"], "hovertemplate": "job=job1<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job1", "marker": {"color": "#EF553B", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job1", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "ALhMCgCEDCQAXCYFAJh/MwDgMikAKOYeAHCZFAAo5h4AuEwKAHCZFACYfzMAcJkUAHCZFAAUcw8APFku"}, "xaxis": "x", "y": ["machine13", "machine11", "machine10", "machine0", "machine2", "machine4", "machine14", "machine7", "machine3", "machine9", "machine12", "machine6", "machine5", "machine1", "machine8"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-19T00:00:00.000000000", "2025-07-08T00:00:00.000000000", "2025-07-24T00:00:00.000000000", "2025-08-22T00:00:00.000000000", "2025-09-02T00:00:00.000000000", "2025-09-28T00:00:00.000000000", "2025-10-14T00:00:00.000000000", "2025-11-04T00:00:00.000000000", "2025-12-02T00:00:00.000000000", "2025-12-08T00:00:00.000000000", "2026-01-23T00:00:00.000000000", "2026-02-27T00:00:00.000000000", "2026-03-13T00:00:00.000000000", "2026-03-22T00:00:00.000000000", "2026-04-08T00:00:00.000000000"], "hovertemplate": "job=job2<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job2", "marker": {"color": "#00cc96", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job2", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AJh/MwA8WS4AmH8zAMy/GQDgMikAuEwKAJh/MwCEDCQAKOYeAJh/MwBwmRQAzL8ZABRzDwBwmRQAFHMP"}, "xaxis": "x", "y": ["machine10", "machine6", "machine3", "machine11", "machine0", "machine2", "machine9", "machine4", "machine12", "machine14", "machine8", "machine5", "machine13", "machine1", "machine7"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-19T00:00:00.000000000", "2025-07-04T00:00:00.000000000", "2025-07-09T00:00:00.000000000", "2025-07-14T00:00:00.000000000", "2025-07-24T00:00:00.000000000", "2025-08-03T00:00:00.000000000", "2025-10-07T00:00:00.000000000", "2025-10-31T00:00:00.000000000", "2025-11-08T00:00:00.000000000", "2025-11-27T00:00:00.000000000", "2025-12-22T00:00:00.000000000", "2026-01-07T00:00:00.000000000", "2026-01-13T00:00:00.000000000", "2026-02-12T00:00:00.000000000", "2026-03-30T00:00:00.000000000"], "hovertemplate": "job=job3<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></ext

## 他のインスタンス

ta50 は最適解は知られていない.

bounds

- upper: 1923
- lower: 1833

In [ ]:
instance_dir = os.path.join(parent, "jsplib/instances")

In [ ]:
fname3 = os.path.join(instance_dir, "ta50")
jobs3 = Job.from_file(fname3)

n=30, m=20


In [ ]:
model3_amplify = ModelAmplifySe(jobs3)
model3_amplify.solve(timeout=10)

mo.md(f"makespan = {model3_amplify.get_makespan()}")

makespan = 2116

In [ ]:
plot_plotly(model3_amplify.to_df())

<marimo-plotly data-figure='{"data": [{"base": ["2025-06-19T00:00:00.000000000", "2025-12-10T00:00:00.000000000", "2026-02-15T00:00:00.000000000", "2026-04-23T00:00:00.000000000", "2026-07-02T00:00:00.000000000", "2026-08-14T00:00:00.000000000", "2026-08-31T00:00:00.000000000", "2026-10-02T00:00:00.000000000", "2026-12-11T00:00:00.000000000", "2027-03-11T00:00:00.000000000", "2027-05-06T00:00:00.000000000", "2027-07-28T00:00:00.000000000", "2027-10-02T00:00:00.000000000", "2027-12-15T00:00:00.000000000", "2028-01-08T00:00:00.000000000", "2028-04-02T00:00:00.000000000", "2028-05-19T00:00:00.000000000", "2028-08-15T00:00:00.000000000", "2030-11-13T00:00:00.000000000", "2030-12-30T00:00:00.000000000"], "hovertemplate": "job=job0<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job0", "marker": {"color": "#636efa", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job0", "orientation": "h", "showlegend": true, "textposition": "auto", "x": [8294400000, 2246400000, 2851200000, 1641600000, 3715200000, 1468800000, 2246400000, 5702400000, 7257600000, 4838400000, 7171200000, 5702400000, 6393600000, 2073600000, 7344000000, 4060800000, 7603200000, 8380800000, 3542400000, 6652800000], "xaxis": "x", "y": ["machine2", "machine8", "machine6", "machine0", "machine4", "machine15", "machine16", "machine13", "machine5", "machine12", "machine10", "machine7", "machine14", "machine19", "machine1", "machine3", "machine18", "machine17", "machine9", "machine11"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-19T00:00:00.000000000", "2025-10-25T00:00:00.000000000", "2025-12-16T00:00:00.000000000", "2026-03-24T00:00:00.000000000", "2026-07-10T00:00:00.000000000", "2026-08-29T00:00:00.000000000", "2027-04-11T00:00:00.000000000", "2027-11-04T00:00:00.000000000", "2028-08-13T00:00:00.000000000", "2028-11-20T00:00:00.000000000", "2029-05-14T00:00:00.000000000", "2029-09-02T00:00:00.000000000", "2029-09-12T00:00:00.000000000", "2029-11-17T00:00:00.000000000", "2030-02-04T00:00:00.000000000", "2030-03-27T00:00:00.000000000", "2030-04-11T00:00:00.000000000", "2030-08-12T00:00:00.000000000", "2030-09-23T00:00:00.000000000", "2030-12-25T00:00:00.000000000"], "hovertemplate": "job=job1<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job1", "marker": {"color": "#EF553B", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job1", "orientation": "h", "showlegend": true, "textposition": "auto", "x": [6048000000, 3974400000, 7776000000, 5270400000, 2073600000, 5443200000, 8208000000, 2937600000, 4060800000, 4320000000, 5356800000, 864000000, 5702400000, 4492800000, 4233600000, 345600000, 8121600000, 3283200000, 8035200000, 7257600000], "xaxis": "x", "y": ["machine0", "machine10", "machine13", "machine3", "machine2", "machine7", "machine1", "machine16", "machine9", "machine17", "machine18", "machine15", "machine11", "machine8", "machine19", "machine5", "machine4", "machine12", "machine14", "machine6"], "yaxis": "y", "type": "bar"}, {"base": ["2025-08-28T00:00:00.000000000", "2026-03-14T00:00:00.000000000", "2026-06-06T00:00:00.000000000", "2026-12-29T00:00:00.000000000", "2027-01-23T00:00:00.000000000", "2028-10-21T00:00:00.000000000", "2029-01-28T00:00:00.000000000", "2029-03-25T00:00:00.000000000", "2029-04-26T00:00:00.000000000", "2029-05-27T00:00:00.000000000", "2029-07-02T00:00:00.000000000", "2029-09-14T00:00:00.000000000", "2029-12-10T00:00:00.000000000", "2030-03-11T00:00:00.000000000", "2030-04-09T00:00:00.000000000", "2030-06-01T00:00:00.000000000", "2030-07-21T00:00:00.000000000", "2030-10-04T00:00:00.000000000", "2030-11-09T00:00:00.000000000", "2030-12-12T00:00:00.000000000"], "hovertemplate": "job=job2<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job2", "marker": {"color": "#00cc96", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job2", "orientation": "h", "showlegend": true, "textposition": "auto", "x": [4147200000, 5184000000, 1296000000, 2160000000, 

In [ ]:
model3_cpsat = ModelCpSat(jobs3)
model3_cpsat.solve(timeout=10)

mo.md(f"makespan = {round(model3_cpsat.solver.objective_value)}")


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 10 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x33f82f20921558d4)
#Variables: 601 (#ints: 1 in objective) (600 primary variables)
  - 601 in [0,30657]
#kInterval: 600
#kLinMax: 1 (#expressions: 30)
#kLinear2: 570
#kNoOverlap: 20 (#intervals: 600)

Starting presolve at 0.00s
  1.17e-04s  0.00e+00d  [DetectDominanceRelations] 
  7.35e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=21 #num_dual_strengthening=1 
  4.04e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  2.61e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.40e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'392 nodes and 2'971 arcs.
[Symmetry] Symmetry computation done. time: 0.000201232 dtime: 0.0002559


  1.48e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.10e-03s  4.72e-06d  [Probe] #new_bounds=1 
  3.18e-06s  0.00e+00d  [MaxClique] 
  9.81e-05s  0.00e+00d  [DetectDominanceRelations] 
  8.55e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  7.04e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.42e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.70e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  6.18e-05s  3.60e-06d  [DetectDominatedLinearConstraints] #relevant_constraints=600 
  6.98e-05s  0.00e+00d  [DetectDifferentVariables] 
  3.46e-06s  0.00e+00d  [ProcessSetPPC] 
  3.54e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  2.15e-05s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  2.61e-05s  2.13e-05d  [FindBigVerticalLinearOverlap] 
  2.60e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  2.52e-06s  0.00e+00d  [MergeClauses] 
  9.67e-05s  0.00e+00d  [DetectDominanceRelations] 
  7.9

#1       0.05s best:2492  next:[1267,2491] no_lp
#Bound   0.05s best:2492  next:[1804,2491] no_lp


#2       0.09s best:2402  next:[1804,2401] rnd_cst_lns (d=5.00e-01 s=14 t=0.10 p=0.00 stall=0 h=base)
#3       0.10s best:2371  next:[1804,2370] graph_var_lns (d=5.00e-01 s=15 t=0.10 p=0.00 stall=0 h=base)


#4       0.19s best:2366  next:[1804,2365] fixed


...

#120     8.39s best:2160  next:[1811,2159] no_lp


#121     8.53s best:2159  next:[1811,2158] no_lp


#122     9.17s best:2158  next:[1811,2157] no_lp


#123     9.37s best:2157  next:[1811,2156] quick_restart_no_lp



Task timing                                  n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                       'default_lp':         1 [   9.98s,    9.98s]    9.98s   0.00ns    9.98s         1 [   1.04s,    1.04s]    1.04s   0.00ns    1.04s
                 'feasibility_pump':         3 [300.39us,   5.08ms]   2.95ms   1.99ms   8.85ms         2 [ 82.54us, 937.32us] 509.93us 427.39us   1.02ms
                            'fixed':         1 [   9.98s,    9.98s]    9.98s   0.00ns    9.98s         1 [741.84ms, 741.84ms] 741.84ms   0.00ns 741.84ms
                               'fj':         1 [ 47.03ms,  47.03ms]  47.03ms   0.00ns  47.03ms         1 [100.13ms, 100.13ms] 100.13ms   0.00ns 100.13ms
                               'fj':         1 [ 71.84ms,  71.84ms]  71.84ms   0.00ns  71.84ms         1 [100.14ms, 100.14ms] 100.14ms   0.00ns 100.14ms
                        'fs_random':         1 [ 34.36ms,  34.36ms]  34.36ms   0.

In [ ]:
plot_plotly(model3_cpsat.to_df())

<marimo-plotly data-figure='{"data": [{"base": ["2025-06-20T00:00:00.000000000", "2025-10-05T00:00:00.000000000", "2026-01-25T00:00:00.000000000", "2026-05-19T00:00:00.000000000", "2026-07-09T00:00:00.000000000", "2026-08-21T00:00:00.000000000", "2026-10-06T00:00:00.000000000", "2026-11-29T00:00:00.000000000", "2027-02-07T00:00:00.000000000", "2027-07-09T00:00:00.000000000", "2028-05-31T00:00:00.000000000", "2029-01-16T00:00:00.000000000", "2029-04-13T00:00:00.000000000", "2029-06-27T00:00:00.000000000", "2029-07-21T00:00:00.000000000", "2029-10-17T00:00:00.000000000", "2029-12-03T00:00:00.000000000", "2030-03-27T00:00:00.000000000", "2030-12-02T00:00:00.000000000", "2031-01-12T00:00:00.000000000"], "hovertemplate": "job=job0<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job0", "marker": {"color": "#636efa", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job0", "orientation": "h", "showlegend": true, "textposition": "auto", "x": [8294400000, 2246400000, 2851200000, 1641600000, 3715200000, 1468800000, 2246400000, 5702400000, 7257600000, 4838400000, 7171200000, 5702400000, 6393600000, 2073600000, 7344000000, 4060800000, 7603200000, 8380800000, 3542400000, 6652800000], "xaxis": "x", "y": ["machine2", "machine8", "machine6", "machine0", "machine4", "machine15", "machine16", "machine13", "machine5", "machine12", "machine10", "machine7", "machine14", "machine19", "machine1", "machine3", "machine18", "machine17", "machine9", "machine11"], "yaxis": "y", "type": "bar"}, {"base": ["2025-06-19T00:00:00.000000000", "2025-09-15T00:00:00.000000000", "2025-11-01T00:00:00.000000000", "2026-01-30T00:00:00.000000000", "2026-04-01T00:00:00.000000000", "2026-05-21T00:00:00.000000000", "2026-12-15T00:00:00.000000000", "2027-08-06T00:00:00.000000000", "2027-09-09T00:00:00.000000000", "2027-10-26T00:00:00.000000000", "2027-12-17T00:00:00.000000000", "2028-02-17T00:00:00.000000000", "2029-06-12T00:00:00.000000000", "2029-08-17T00:00:00.000000000", "2029-11-01T00:00:00.000000000", "2029-12-20T00:00:00.000000000", "2029-12-24T00:00:00.000000000", "2030-06-13T00:00:00.000000000", "2030-11-17T00:00:00.000000000", "2031-02-18T00:00:00.000000000"], "hovertemplate": "job=job1<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job1", "marker": {"color": "#EF553B", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job1", "orientation": "h", "showlegend": true, "textposition": "auto", "x": [6048000000, 3974400000, 7776000000, 5270400000, 2073600000, 5443200000, 8208000000, 2937600000, 4060800000, 4320000000, 5356800000, 864000000, 5702400000, 4492800000, 4233600000, 345600000, 8121600000, 3283200000, 8035200000, 7257600000], "xaxis": "x", "y": ["machine0", "machine10", "machine13", "machine3", "machine2", "machine7", "machine1", "machine16", "machine9", "machine17", "machine18", "machine15", "machine11", "machine8", "machine19", "machine5", "machine4", "machine12", "machine14", "machine6"], "yaxis": "y", "type": "bar"}, {"base": ["2025-12-11T00:00:00.000000000", "2026-03-01T00:00:00.000000000", "2026-07-01T00:00:00.000000000", "2027-10-28T00:00:00.000000000", "2027-11-22T00:00:00.000000000", "2028-05-27T00:00:00.000000000", "2028-12-28T00:00:00.000000000", "2029-03-26T00:00:00.000000000", "2029-05-25T00:00:00.000000000", "2029-06-26T00:00:00.000000000", "2029-09-04T00:00:00.000000000", "2029-12-05T00:00:00.000000000", "2030-02-15T00:00:00.000000000", "2030-05-17T00:00:00.000000000", "2030-07-22T00:00:00.000000000", "2030-08-25T00:00:00.000000000", "2030-11-17T00:00:00.000000000", "2030-12-08T00:00:00.000000000", "2031-01-13T00:00:00.000000000", "2031-01-22T00:00:00.000000000"], "hovertemplate": "job=job2<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job2", "marker": {"color": "#00cc96", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job2", "orientation": "h", "showlegend": true, "textposition": "auto", "x": [4147200000, 5184000000, 1296000000, 2160000000, 

## didp での求解

### 状態

- $\text{Q}$: set 変数. 配置されていないタスクの集合を表す.
- $\text{tm}_m \space (\forall m: \text{machine})$: 機械ごとに makespan を保持する.
- $\text{tj}_j \space (\forall j: \text{job})$: ジョブごとに makespan を保持する.

### 目的関数

- $\text{makespan} := \max \{ \text{tm}_m, \text{tj}_j \mid m: \text{machine}, \space j: \text{job} \}$

### 更新規則

- タスク $\text{task} \in Q$ は全ての先行タスクが $Q$ に属していない時配置可能.
- $\text{task}$ が配置された場合, それを $Q$ から取り除く.
- $\text{task}$ が配置された場合, タスクを処理する機械 $m$ とタスクの属するジョブ $j$ に対して以下のように更新する.
    - $\text{tm}_m \leftarrow \max(\text{tm}_m + t_\text{task}, \space \text{tj}_j + t_\text{task})$
    - $\text{tj}_j \leftarrow \max(\text{tm}_m + t_\text{task}, \space \text{tj}_j + t_\text{task})$
- 上記更新の後, 目的関数を再計算する.

In [ ]:
class ModelDidp:
    def __init__(self, jobs: list[Job]):
        self.jobs = jobs
        n_tasks = sum(len(job.tasks) for job in self.jobs)
        n_machines = len(
            set(task.machine for job in self.jobs for task in job.tasks)
        )

        self.model = didppy.Model()

        objtype_task = self.model.add_object_type(number=n_tasks)

        remaining = self.model.add_set_var(
            object_type=objtype_task, target=list(range(n_tasks))
        )

        cur_time_per_machine = [
            # self.model.add_int_var(target=0) for _ in range(n_machines)
            self.model.add_int_resource_var(target=0, less_is_better=True)
            for _ in range(n_machines)
        ]
        cur_time_per_job = [
            # self.model.add_int_var(target=0) for _ in self.jobs
            self.model.add_int_resource_var(target=0, less_is_better=True)
            for _ in self.jobs
        ]

        self.model.add_base_case([remaining.is_empty()])

        # task_to_time = self.model.add_int_table(
        #     [task.time for job in self.jobs for task in job.tasks]
        # )
        # task_to_machine = self.model.add_int_table(
        #     [task.machine for job in self.jobs for task in job.tasks]
        # )

        precs = []
        id_jobtask = 0
        for id_job, job in enumerate(self.jobs):
            prec = set()
            for id_task, task in enumerate(job.tasks):
                precs.append(prec.copy())
                prec.add(id_jobtask)
                id_jobtask += 1

        task_to_prec = self.model.add_set_table(
            precs, object_type=objtype_task
        )

        id_jobtask = 0
        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                sched = didppy.Transition(
                    name=f"sched_job{id_job}_task{id_task}",
                    cost=(
                        didppy.max(
                            didppy.IntExpr.state_cost(),
                            didppy.max(
                                cur_time_per_machine[task.machine] + task.time,
                                cur_time_per_job[id_job] + task.time,
                            ),
                        )
                    ),
                    effects=[
                        (remaining, remaining.remove(id_jobtask)),
                        (
                            cur_time_per_job[id_job],
                            didppy.max(
                                cur_time_per_machine[task.machine] + task.time,
                                cur_time_per_job[id_job] + task.time,
                            ),
                        ),
                        (
                            cur_time_per_machine[task.machine],
                            didppy.max(
                                cur_time_per_machine[task.machine] + task.time,
                                cur_time_per_job[id_job] + task.time,
                            ),
                        ),
                    ],
                    preconditions=[
                        remaining.contains(id_jobtask),
                        remaining.isdisjoint(task_to_prec[id_jobtask]),
                    ],
                )
                self.model.add_transition(sched)

                id_jobtask += 1

        task_to_min_cost = []
        for job in self.jobs:
            cost = sum(task.time for task in job.tasks)
            for task in job.tasks:
                task_to_min_cost.append(cost)
                cost -= task.time
        task_to_min_cost_table = self.model.add_int_table(task_to_min_cost)
        self.model.add_dual_bound(
            remaining.is_empty().if_then_else(0, task_to_min_cost_table.min(remaining))
        )

    def solve(self, timeout=10, threads: int = 8) -> None:
        self.solver = didppy.CABS(
            self.model, threads=threads, quiet=False, time_limit=timeout
        )
        # self.solver = didppy.LNBS(
        #     self.model, threads=threads, quiet=False, time_limit=timeout
        # )
        self.solution: didppy.Solution = self.solver.search()

In [ ]:
model1_didp = ModelDidp(jobs1)
model1_didp.solve(threads=10, timeout=10)

mo.md(f"makespan = {round(model1_didp.solution.cost)}")

Solver: CABS from DIDPPy v0.9.0
Searched with beam size: 1, threads: 10, kept: 174, sent: 0
Searched with beam size: 1, expanded: 36, elapsed time: 0.001700887
New primal bound: 84, expanded: 36, elapsed time: 0.001703622
Searched with beam size: 2, threads: 10, kept: 165, sent: 182
Searched with beam size: 2, expanded: 105, elapsed time: 0.002063455
New dual bound: 4, expanded: 105, elapsed time: 0.002064767
New primal bound: 77, expanded: 105, elapsed time: 0.002082311
Searched with beam size: 4, threads: 10, kept: 169, sent: 521
Searched with beam size: 4, expanded: 240, elapsed time: 0.002601706
New primal bound: 65, expanded: 240, elapsed time: 0.002634638
Searched with beam size: 8, threads: 10, kept: 179, sent: 1149
Searched with beam size: 8, expanded: 492, elapsed time: 0.003257901
Searched with beam size: 16, threads: 10, kept: 241, sent: 2294
Searched with beam size: 16, expanded: 983, elapsed time: 0.00417466
New dual bound: 6, expanded: 983, elapsed time: 0.004177385
Searc

In [ ]:
model1_didp.solution.is_optimal

False

In [ ]:
for _t in model1_didp.solution.transitions:
    print(_t.name)

sched_job2_task0
sched_job0_task0
sched_job1_task0
sched_job1_task1
sched_job4_task0
sched_job2_task1
sched_job5_task0
sched_job0_task1
sched_job5_task1
sched_job3_task0
sched_job2_task2
sched_job3_task1
sched_job5_task2
sched_job0_task2
sched_job3_task2
sched_job1_task2
sched_job4_task1
sched_job2_task3
sched_job4_task2
sched_job2_task4
sched_job0_task3
sched_job1_task3
sched_job3_task3
sched_job2_task5
sched_job5_task3
sched_job4_task3
sched_job0_task4
sched_job3_task4
sched_job5_task4
sched_job1_task4
sched_job1_task5
sched_job3_task5
sched_job0_task5
sched_job4_task4
sched_job4_task5
sched_job5_task5


In [ ]:
model2_didp = ModelDidp(jobs2)
model2_didp.solve(threads=10, timeout=10)

mo.md(f"makespan = {round(model2_didp.solution.cost)}")

Solver: CABS from DIDPPy v0.9.0
Searched with beam size: 1, threads: 10, kept: 23775, sent: 0
Searched with beam size: 1, expanded: 675, elapsed time: 0.034590592
New primal bound: 370, expanded: 675, elapsed time: 0.034592966
Searched with beam size: 2, threads: 10, kept: 23733, sent: 23575
Searched with beam size: 2, expanded: 2021, elapsed time: 0.075615247
New dual bound: 2, expanded: 2021, elapsed time: 0.075616519
New primal bound: 367, expanded: 2021, elapsed time: 0.076169899
Searched with beam size: 4, threads: 10, kept: 23741, sent: 71009
Searched with beam size: 4, expanded: 4708, elapsed time: 0.12512166
New primal bound: 353, expanded: 4708, elapsed time: 0.125813412
Searched with beam size: 8, threads: 10, kept: 23755, sent: 164220
Searched with beam size: 8, expanded: 10069, elapsed time: 0.207351483
Searched with beam size: 16, threads: 10, kept: 38087, sent: 339766
Searched with beam size: 16, expanded: 20795, elapsed time: 0.364703437
New primal bound: 343, expanded: 

In [ ]:
model3_didp = ModelDidp(jobs3)
model3_didp.solve(threads=10, timeout=10)

mo.md(f"makespan = {round(model3_didp.solution.cost)}")

Solver: CABS from DIDPPy v0.9.0
Searched with beam size: 1, threads: 10, kept: 15086, sent: 0
Searched with beam size: 1, expanded: 600, elapsed time: 0.023599223
New primal bound: 2756, expanded: 600, elapsed time: 0.023601978
Searched with beam size: 2, threads: 10, kept: 15864, sent: 15790
Searched with beam size: 2, expanded: 1798, elapsed time: 0.052234147
New dual bound: 12, expanded: 1798, elapsed time: 0.05223566
New primal bound: 2612, expanded: 1798, elapsed time: 0.05284758
Searched with beam size: 4, threads: 10, kept: 15529, sent: 46409
Searched with beam size: 4, expanded: 4172, elapsed time: 0.086004753
Searched with beam size: 8, threads: 10, kept: 15816, sent: 110259
Searched with beam size: 8, expanded: 8955, elapsed time: 0.140949471
New primal bound: 2512, expanded: 8955, elapsed time: 0.141479276
Searched with beam size: 16, threads: 10, kept: 25300, sent: 225307
Searched with beam size: 16, expanded: 18451, elapsed time: 0.248606915
Searched with beam size: 32, th

## 離接定式化のグラフによる表示

JSP は タスクをノード, 依存関係や同時処理禁止規則をエッジで表現したグラフからエッジを選択する問題として表現することができる.

### 参考

- https://acrogenesis.com/or-tools/documentation/user_manual/manual/ls/jobshop_def_data.html
- https://zenn.dev/fusic/articles/0fed6d5dfbdeb5

### 定数

- $J$: ジョブの集合
- $M$: マシンの集合
- $O$: オペレーションの集合
- $O_j$: ジョブ $j$ のオペレーションの集合
- $O_m$: マシン $m$ で処理するオペレーションの集合
- $t_o$: オペレーション $o$ の処理時間

### グラフ

- ノード $N := O \cup \{ \text{source}, \text{target} \}$
- エッジ $E := E^c \cup E^d$
    - Conjunctive Edges $E^c$: オペレーション $o$ と $o'$ が同じジョブに属しており, $o$ の後に $o'$ を処理しなければならない場合, $(o, o') \in E^c$.
      また, $o$ があるジョブの最初のオペレーションであるとき $(\text{source}, o) \in E^c$.
      $o$ があるジョブの最後のオペレーションであるとき $(o, \text{target}) \in E^c$.
    - Disjunctive Edges $E^d$: オペレーション $o$ と $o'$ が同じマシンで処理されるとき, $(o, o') \in E^d$ かつ $(o', o) \in E^d$.
      このエッジは双方向のうちどちらかを選択し, 選択されたエッジによりオペレーションの処理順序が定まる.

このグラフのエッジで繋がれたノード(オペレーション)の間には処理順序の関係がある.
Conjunctive edge は同一ジョブ内オペレーションの順序関係を表し,
Disjunctive edge は同一マシンで処理するオペレーションの間の順序関係を表す.

### 決定変数

- $x_e \in \{ 0, 1 \} \space (e \in E)$: エッジ $e$ を選択する場合のみ $1$.
- $s_n \in \mathbb{Z} \space (n \in N)$: オペレーションの開始時刻. $\text{source}$ ノードの開始時刻は 0, 処理時間も 0 とする.

### 制約条件

- $e = (u, v)$ とする. このとき $x_e = 1 \Rightarrow s_u + t_u \leq s_v$
    - $e \in E^c \Rightarrow x_e = 1$
    - $(u, v) \in E^d \Rightarrow x_{(u,v)} + x_{(v,u)} = 1$

### 目的関数

- $s_\text{target}$ が makespan を表す. これを最小化する.

## 巡回路制約を用いた実装

上記のグラフにマシン自体をノードとして足し,
disjunctive edge のみを辿ってマシンごとに順回路を作成することでマシン内での実行順を記述することができる.

In [ ]:
class ModelCpSatArc:
    def __init__(self, jobs: list[Job]):
        machines = sorted(
            list(set(task.machine for job in jobs for task in job.tasks))
        )

        # タスクに 0 から番号を割り振る.
        # source と target はそれぞれ -1, -2 とする.
        task_indices = []
        idx = 0
        for job in jobs:
            indices = []
            for task in job.tasks:
                indices.append(idx)
                idx += 1
            task_indices.append(indices)

        all_tasks = [task for job in jobs for task in job.tasks]

        horizon = sum(task.time for job in jobs for task in job.tasks)

        model = cp_model.CpModel()

        edges = {}
        starts = {}

        # start time
        starts[-1] = model.new_constant(0)
        starts[-2] = model.new_int_var(0, horizon, "")
        for indices in task_indices:
            for idx in indices:
                starts[idx] = model.new_int_var(0, horizon, "")

        # Conjunctive Edges
        for indices in task_indices:
            edges[(-1, indices[0])] = model.new_constant(1)
            edges[(indices[-1], -2)] = model.new_constant(1)
            for i, _ in enumerate(indices):
                if i == 0:
                    continue
                edges[(indices[i - 1], indices[i])] = model.new_constant(1)

        # Disjunctive Edges
        for m in machines:
            indices_m = [
                idx
                for indices, job in zip(task_indices, jobs)
                for idx, task in zip(indices, job.tasks)
                if task.machine == m
            ] + [-3 - m]

            edges_m = {
                (u, v): model.new_bool_var("")
                for u in indices_m
                for v in indices_m
                if u != v
            }
            model.add_circuit((u, v, var) for (u, v), var in edges_m.items())

            edges |= edges_m

        for (u, v), var in edges.items():
            if u < -2 or v < -2:
                continue

            if u == -1:
                model.add(starts[u] <= starts[v])
            else:
                model.add(
                    starts[u] + all_tasks[u].time <= starts[v]
                ).only_enforce_if(edges[(u, v)])

        model.minimize(starts[-2])

        self.model = model
        self.objective = starts[-2]

    def solve(self, timeout: int = 10):
        self.solver = cp_model.CpSolver()
        self.solver.parameters.log_search_progress = True
        self.solver.parameters.max_time_in_seconds = timeout
        self.status = self.solver.solve(self.model)

In [ ]:
model1_cpsatarc = ModelCpSatArc(jobs1)
model1_cpsatarc.solve()

mo.md(f"makespan = {round(model1_cpsatarc.solver.objective_value)}")


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 10 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xaacdc7c50f505cd1)
#Variables: 291 (#ints: 1 in objective) (291 primary variables)
  - 252 Booleans in [0,1]
  - 37 in [0,197]
  - 2 constants in {0,1} 
#kCircuit: 6
#kLinear2: 222 (#enforced: 216)

Starting presolve at 0.00s
  4.80e-05s  0.00e+00d  [DetectDominanceRelations] 
  1.52e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=6 #num_dual_strengthening=1 
  1.62e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  8.09e-06s  0.00e+00d  [DetectDuplicateColumns] 
  1.24e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 836 nodes and 1'403 arcs.
[Symmetry] Symmetry computation done. time: 8.6504e-05 dtime: 0.00010138
  1.29e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.40e-03s  6.08e-04d  [Probe] #probed=504 #new_binary_clauses=126 
  2.18e-06s  0.00

  1.26e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.26e-03s  6.10e-04d  [Probe] #probed=504 #new_binary_clauses=36 
  6.49e-05s  4.58e-05d  [MaxClique] 
  5.03e-05s  0.00e+00d  [DetectDominanceRelations] 
  5.94e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  3.51e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.24e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  8.63e-06s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  7.45e-06s  2.16e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=36 
  9.13e-05s  0.00e+00d  [DetectDifferentVariables] #different=36 
  2.14e-05s  5.40e-07d  [ProcessSetPPC] #relevant_constraints=90 
  3.18e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  2.87e-05s  1.98e-05d  [FindBigAtMostOneAndLinearOverlap] 
  3.23e-05s  1.92e-05d  [FindBigVerticalLinearOverlap] 
  2.64e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  3.98e-06s  0.00e+00d  [MergeClauses]

#2       0.02s best:139   next:[47,138]   quick_restart
#3       0.02s best:130   next:[47,129]   default_lp
#4       0.02s best:128   next:[47,127]   no_lp
#5       0.02s best:122   next:[47,121]   no_lp
#6       0.02s best:98    next:[47,97]    no_lp
#7       0.03s best:95    next:[47,94]    no_lp
#8       0.03s best:93    next:[47,92]    no_lp
#9       0.03s best:91    next:[47,90]    quick_restart_no_lp
#10      0.03s best:90    next:[47,89]    no_lp
#11      0.03s best:89    next:[47,88]    no_lp
#12      0.03s best:88    next:[47,87]    no_lp
#13      0.03s best:87    next:[47,86]    quick_restart_no_lp
#14      0.03s best:86    next:[47,85]    no_lp
#15      0.03s best:84    next:[47,83]    no_lp
#16      0.03s best:83    next:[47,82]    no_lp
#17      0.03s best:82    next:[47,81]    no_lp
#18      0.03s best:77    next:[47,76]    no_lp
#19      0.03s best:76    next:[47,75]    no_lp
#20      0.03s best:75    next:[47,74]    no_lp
#21      0.03s best:73    next:[47,72]    no_lp

#Model   0.03s var:280/289 constraints:375/402
#31      0.04s best:63    next:[47,62]    quick_restart_no_lp
#32      0.04s best:62    next:[47,61]    quick_restart_no_lp
#33      0.04s best:61    next:[47,60]    quick_restart_no_lp
#34      0.04s best:60    next:[47,59]    no_lp
#35      0.04s best:59    next:[47,58]    no_lp
#Model   0.04s var:271/289 constraints:349/402
#36      0.04s best:58    next:[47,57]    no_lp
#37      0.04s best:57    next:[47,56]    no_lp
#Model   0.04s var:263/289 constraints:328/402
#Model   0.04s var:259/289 constraints:315/402
#38      0.04s best:56    next:[47,55]    quick_restart
#39      0.04s best:55    next:[47,54]    no_lp


#Model   0.05s var:244/289 constraints:295/402
#Model   0.05s var:221/289 constraints:260/402
#Bound   0.05s best:55    next:[49,54]    quick_restart_no_lp
#Done    0.05s quick_restart_no_lp

Task timing                        n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
             'default_lp':         1 [ 34.53ms,  34.53ms]  34.53ms   0.00ns  34.53ms         1 [  5.37ms,   5.37ms]   5.37ms   0.00ns   5.37ms
       'feasibility_pump':         1 [  1.02ms,   1.02ms]   1.02ms   0.00ns   1.02ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                     'fj':         1 [  5.75ms,   5.75ms]   5.75ms   0.00ns   5.75ms         1 [  4.79ms,   4.79ms]   4.79ms   0.00ns   4.79ms
                     'fj':         1 [ 16.17ms,  16.17ms]  16.17ms   0.00ns  16.17ms         1 [ 15.20ms,  15.20ms]  15.20ms   0.00ns  15.20ms
              'fs_random':         1 [  4.62ms,   4.62ms]   4.62ms   0.00ns   4.62ms         

In [ ]:
model3_cpsatarc = ModelCpSatArc(jobs3)
model3_cpsatarc.solve(timeout=10)

mo.md(f"makespan = {round(model3_cpsatarc.solver.objective_value)}")


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 10 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x50689656cfb32057)
#Variables: 19'203 (#ints: 1 in objective) (19'203 primary variables)
  - 18'600 Booleans in [0,1]
  - 601 in [0,30657]
  - 2 constants in {0,1} 
#kCircuit: 20
#kLinear2: 18'030 (#enforced: 18'000)

Starting presolve at 0.00s


  3.56e-03s  0.00e+00d  [DetectDominanceRelations] 
  2.34e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=20 #num_dual_strengthening=1 
  6.67e-05s  0.00e+00d  [ExtractEncodingFromLinear] 
  7.38e-04s  0.00e+00d  [DetectDuplicateColumns] 
  4.57e-04s  0.00e+00d  [DetectDuplicateConstraints] 


[Symmetry] Graph for symmetry has 57'022 nodes and 109'799 arcs.
[Symmetry] Symmetry computation done. time: 0.00541642 dtime: 0.00701877


  5.36e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  3.54e-01s  1.85e-01d  [Probe] #probed=37'200 #new_binary_clauses=9'300 
  6.20e-05s  0.00e+00d  [MaxClique] 


  2.88e-03s  0.00e+00d  [DetectDominanceRelations] 
  4.50e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  2.06e-03s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  4.90e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  4.66e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.17e-04s  3.60e-06d  [DetectDominatedLinearConstraints] #relevant_constraints=600 
  5.09e-03s  0.00e+00d  [DetectDifferentVariables] #different=600 
  1.29e-04s  0.00e+00d  [ProcessSetPPC] 
  1.47e-04s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  8.54e-04s  2.61e-04d  [FindBigAtMostOneAndLinearOverlap] 


...

#Bound   8.27s best:inf   next:[1252,30657] reduced_costs


#1       8.32s best:24255 next:[1252,24254] no_lp


#2       8.67s best:24254 next:[1252,24253] graph_arc_lns (d=5.00e-01 s=235 t=0.10 p=0.00 stall=0 h=base) [hint]


#3       8.86s best:23887 next:[1252,23886] graph_dec_lns (d=5.00e-01 s=237 t=0.10 p=0.00 stall=0 h=base) [hint]



Task timing                        n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
             'default_lp':         1 [   8.23s,    8.23s]    8.23s   0.00ns    8.23s         1 [228.05ms, 228.05ms] 228.05ms   0.00ns 228.05ms
       'feasibility_pump':        60 [  2.49ms, 200.15ms]   7.10ms  26.74ms 425.79ms        59 [625.76us, 117.24ms]   2.60ms  15.05ms 153.53ms
                     'fj':        45 [ 20.05ms, 257.83ms] 136.09ms  85.61ms    6.12s        45 [100.14ms, 111.74ms] 101.31ms   1.86ms    4.56s
                     'fj':        58 [ 19.11ms, 252.99ms] 105.49ms  93.28ms    6.12s        58 [100.14ms, 119.97ms] 101.98ms   3.58ms    5.92s
              'fs_random':         1 [   6.50s,    6.50s]    6.50s   0.00ns    6.50s         1 [ 10.03ms,  10.03ms]  10.03ms   0.00ns  10.03ms
        'fs_random_no_lp':         1 [   6.49s,    6.49s]    6.49s   0.00ns    6.49s         1 [  3.15ms,   3.15ms]   3.15ms   0.00ns   3.15m

なんか全然ダメだった...

区間変数より circuit constraint の方がいい場合もあるらしい[^1]が, 今回はダメそう.

[^1]: https://d-krupke.github.io/cpsat-primer/04B_advanced_modelling.html